In [1]:
import pandas as pd
import numpy as np

def reduce_mem_usage_safe(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire initial du DataFrame: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        if col_type == 'bool':
            df[col] = df[col].astype('bool')
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)
        elif str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire final du DataFrame: {end_mem:.2f} MB")
    print(f"Mémoire réduite de {(start_mem - end_mem) / start_mem * 100:.1f} %")

    return df


In [2]:
def reduce_mem_usage(df):
    """
    Itère sur toutes les colonnes d'un DataFrame et réduit la précision
    des types numériques (int et float) pour diminuer la consommation de mémoire.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire initial du DataFrame: {start_mem:.2f} MB")

    for col in df.columns:
        col_type = df[col].dtype

        # Traiter uniquement les colonnes numériques
        if col_type != object and col_type != str and col_type != bool:
            c_min = df[col].min()
            c_max = df[col].max()

            # --- Conversion des entiers (Integers) ---
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # --- Conversion des décimaux (Floats) ---
            else:
                # La majorité de vos colonnes d'agrégats (mean, var, proportions) sont ici
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    # Conversion principale : float64 -> float32
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64) # Garder float64 si la précision est nécessaire

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"Usage mémoire final du DataFrame: {end_mem:.2f} MB")
    print(f"Mémoire réduite de {(start_mem - end_mem) / start_mem * 100:.1f} %")

    return df

ETAPE 1
netoyage et imputation 

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm  # Pour les barres de progression


# --- 1. Optimisation Mémoire ---
def reduce_mem_usage_safe(df, verbose=True):
    """
    Itère sur toutes les colonnes d'un DataFrame et modifie les types de données
    pour minimiser l'utilisation de la mémoire.
    """
    df_out = df.copy()  # Evite les SettingWithCopyWarning
    start_mem = df_out.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage of dataframe before: {start_mem:.2f} MB')

    for col in tqdm(df_out.columns, desc="Reducing memory"):
        col_type = df_out[col].dtype

        if col_type != object:
            c_min = df_out[col].min()
            c_max = df_out[col].max()
            if str(col_type)[:3] == 'int':
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df_out[col] = df_out[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df_out[col] = df_out[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df_out[col] = df_out[col].astype(np.int32)
                else:
                    df_out[col] = df_out[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df_out[col] = df_out[col].astype(np.float32)
                else:
                    df_out[col] = df_out[col].astype(np.float64)

    end_mem = df_out.memory_usage().sum() / 1024**2
    if verbose:
        print(f'Memory usage after optimization: {end_mem:.2f} MB ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')
    return df_out


# --- 2. Analyse des NaN ---
def get_missing_values_table(df, df_name, top_n=20):
    """Calcule et affiche le tableau des valeurs manquantes pour un DataFrame."""
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * mis_val / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: 'Missing Values', 1: '% of Total Values'}
    )
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:, 1] != 0
    ].sort_values('% of Total Values', ascending=False).round(1)

    print(f"\n--- Missing values for {df_name} ---")
    print(f"DataFrame has {df.shape[1]} columns.")
    print(f"There are {mis_val_table_ren_columns.shape[0]} columns with missing values.")

    return mis_val_table_ren_columns.head(top_n)


# --- 3. Définition des chemins et chargement des fichiers ---
DATA_DIR = r"C:\Users\maill\OneDrive\Bureau\majeur_ia\dataenginnering\projet\home-credit-default-risk"

csv_files = {
    'train': 'application_train.csv',
    'test': 'application_test.csv',
    'bureau': 'bureau.csv',
    'bureau_balance': 'bureau_balance.csv',
    'credit_card_balance': 'credit_card_balance.csv',
    'installments_payments': 'installments_payments.csv',
    'POS_CASH_balance': 'POS_CASH_balance.csv',
    'previous_application': 'previous_application.csv'
}

dataframes = {}

print("Starting to load and optimize dataframes...")
for name, filename in csv_files.items():
    file_path = os.path.join(DATA_DIR, filename)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        dataframes[name] = reduce_mem_usage_safe(df, verbose=True)
    else:
        print(f"File not found: {file_path}. Skipping.")
print("All available dataframes loaded.")


# --- 4. Analyse des NaN pour tous les fichiers ---
for name, df in dataframes.items():
    missing_table = get_missing_values_table(df, name)
    print(missing_table)


# --- 5. Nettoyage et imputation des fichiers principaux ---
df_train = dataframes.get('train')
df_test = dataframes.get('test')

if df_train is not None and df_test is not None:
    print("\n--- Imputation Strategy on Main DataFrames ---")

    house_cols = [
        col for col in df_train.columns
        if ('APARTMENTS' in col or 'YEARS_BUILD' in col or 'COMMONAREA' in col or 'ELEVATORS' in col or 'FONDKAPREMONT' in col)
        and 'MODE' not in col
    ]

    for df in [df_train, df_test]:
        # Imputation variables immobilières
        for col in house_cols:
            if col in df.columns:
                df[f'{col}_IS_NAN'] = df[col].isnull().astype(np.int8)
                df[col].fillna(0, inplace=True)

        # Imputation EXT_SOURCE
        for col in ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']:
            if col in df.columns:
                df[f'{col}_IS_NAN'] = df[col].isnull().astype(np.int8)
                df[col].fillna(df[col].median(), inplace=True)

        # Imputation colonnes catégorielles
        for col in ['NAME_TYPE_SUITE', 'OCCUPATION_TYPE', 'FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE']:
            if col in df.columns and df[col].dtype == object:
                df[col].fillna('Missing', inplace=True)

        # Correction DAYS_EMPLOYED
        if 'DAYS_EMPLOYED' in df.columns:
            df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
            df['DAYS_EMPLOYED'].fillna(df['DAYS_EMPLOYED'].median(), inplace=True)

    print("Main DataFrames (train/test) cleaned and imputed.")
    dataframes['train'] = df_train
    dataframes['test'] = df_test


Starting to load and optimize dataframes...
Memory usage of dataframe before: 286.23 MB


Reducing memory: 100%|██████████| 122/122 [00:00<00:00, 289.83it/s]


Memory usage after optimization: 128.16 MB (55.2% reduction)
Memory usage of dataframe before: 45.00 MB


Reducing memory: 100%|██████████| 121/121 [00:00<00:00, 1628.20it/s]

Memory usage after optimization: 20.27 MB (55.0% reduction)


...

Memory usage after optimization: 318.63 MB (52.7% reduction)
Memory usage of dataframe before: 830.41 MB


Reducing memory: 100%|██████████| 8/8 [00:00<00:00, 12.72it/s]


Memory usage after optimization: 389.25 MB (53.1% reduction)
Memory usage of dataframe before: 610.43 MB


Reducing memory: 100%|██████████| 8/8 [00:00<00:00, 23.51it/s]


Memory usage after optimization: 276.60 MB (54.7% reduction)


1.2 enrichissement de donner 

In [11]:
import pandas as pd
import numpy as np

# Récupération des DataFrames chargés
dataframes = globals().get('dataframes')

if dataframes is None:
    print("Error: DataFrames not loaded. Please ensure the previous block of code was executed.")
    exit() 

df_train = dataframes['train']
df_test = dataframes['test']
df_bureau = dataframes['bureau']
df_bureau_balance = dataframes['bureau_balance']

print("\n--- Starting Feature Engineering: Bureau & Bureau Balance ---")

# --- 1. TRAITEMENT DE BUREAU_BALANCE ---
print("Processing bureau_balance...")

# One-Hot Encoding pour STATUS
df_bureau_balance = pd.get_dummies(df_bureau_balance, dummy_na=False)

if 'DAYS_CREDIT_ENDDATE_FLAG' in df_bureau.columns:
    df_bureau = df_bureau.drop(columns=['DAYS_CREDIT_ENDDATE_FLAG'])

# Fonction d'agrégation corrigée
def group_and_aggregate(df, group_key, df_name, agg_funcs):
    """Effectue l'agrégation et renomme les colonnes."""
    # Agrégation standard
    agg_df = df.groupby(group_key).agg(agg_funcs)
    # Aplatir les noms de colonnes
    agg_df.columns = pd.Index([f'{df_name}_{col[0]}_{col[1].upper()}' for col in agg_df.columns.tolist()])
    return agg_df

# Agrégation bureau_balance par SK_ID_BUREAU
bb_agg_funcs = ['min', 'max', 'count', 'mean']  # <--- corrigé
bb_grouped = group_and_aggregate(
    df_bureau_balance.drop(columns=['MONTHS_BALANCE']),
    group_key='SK_ID_BUREAU',
    df_name='BUREAU_BALANCE',
    agg_funcs=bb_agg_funcs
)

# Fusion dans bureau
df_bureau = df_bureau.merge(bb_grouped, how='left', on='SK_ID_BUREAU')
print(f"Merged bureau_balance into bureau. New bureau shape: {df_bureau.shape}")

# --- 2. TRAITEMENT DE BUREAU ---
print("Processing bureau...")

df_bureau = pd.get_dummies(df_bureau, dummy_na=False)

BUREAU_IGNORE_COLS = ['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE_Active'] 
bureau_cols = [col for col in df_bureau.columns if col not in BUREAU_IGNORE_COLS]

num_agg = ['min', 'max', 'mean', 'sum', 'var']  # <--- corrigé
cat_agg = ['mean', 'sum']

# Définition des agrégations spécifiques
bureau_agg_funcs = {}
for col in bureau_cols:
    if df_bureau[col].dtype != object:
        if 'COUNT' in col or 'SUM' in col or 'STATUS' in col:
            bureau_agg_funcs[col] = cat_agg
        else:
            bureau_agg_funcs[col] = num_agg

# Agrégation bureau par SK_ID_CURR
bureau_grouped = df_bureau.groupby('SK_ID_CURR').agg(bureau_agg_funcs)
bureau_grouped.columns = pd.Index([f'BUREAU_{col[0]}_{col[1].upper()}' for col in bureau_grouped.columns.tolist()])
print(f"Bureau aggregation done. Aggregated shape: {bureau_grouped.shape}")

# --- 3. FUSION DANS LES DATAFRAMES PRINCIPAUX ---
df_train = df_train.merge(bureau_grouped, how='left', on='SK_ID_CURR')
print(f"Merged bureau features into df_train. New shape: {df_train.shape}")

df_test = df_test.merge(bureau_grouped, how='left', on='SK_ID_CURR')
print(f"Merged bureau features into df_test. New shape: {df_test.shape}")

# Mise à jour des DataFrames globaux
dataframes['train'] = df_train
dataframes['test'] = df_test

print("\nFeature Engineering for Bureau & Bureau Balance completed.")
print("The main DataFrames now contain over 300 new features based on credit history.")



--- Starting Feature Engineering: Bureau & Bureau Balance ---
Processing bureau_balance...
Merged bureau_balance into bureau. New bureau shape: (1716428, 49)
Processing bureau...
Bureau aggregation done. Aggregated shape: (305811, 248)
Merged bureau features into df_train. New shape: (307511, 385)
Merged bureau features into df_test. New shape: (48744, 384)

Feature Engineering for Bureau & Bureau Balance completed.
The main DataFrames now contain over 300 new features based on credit history.


enrichissement des donner2 

In [12]:
# Nécessaire pour les opérations de fusion et d'agrégation
import pandas as pd
import numpy as np

# Récupération des DataFrames mis à jour
dataframes = globals().get('dataframes') 

if dataframes is None or 'previous_application' not in dataframes:
    print("Error: DataFrames not loaded or previous_application is missing.")
    exit() 

df_train = dataframes['train']
df_test = dataframes['test']
df_prev = dataframes['previous_application']

print("\n--- Starting Feature Engineering: Previous Application ---")

# --- 1. PRÉ-TRAITEMENT ET FEATURE CREATION ---

print("Processing previous_application...")

# One-Hot Encoding pour les colonnes catégorielles
df_prev = pd.get_dummies(df_prev, dummy_na=False)

# Création de quelques ratios et features clés (bonnes pratiques du métier)
# Ratio Montant du Crédit / Montant Demandé
df_prev['APP_CREDIT_PER_APPLICATION'] = df_prev['AMT_CREDIT'] / df_prev['AMT_APPLICATION']
# Ratio Montant Annuité / Montant Demandé
df_prev['ANNUITY_PER_APPLICATION'] = df_prev['AMT_ANNUITY'] / df_prev['AMT_APPLICATION']
# Ratio Montant Crédit / Montant Annuité
df_prev['CREDIT_PER_ANNUITY'] = df_prev['AMT_CREDIT'] / df_prev['AMT_ANNUITY']
# Ratio Montant Crédit / Durée du prêt (approximative)
df_prev['CREDIT_PER_TERM'] = df_prev['AMT_CREDIT'] / df_prev['CNT_PAYMENT']


# Correction de l'outlier DAYS_FIRST_DRAWING
# La valeur 365243 signifie ici "Inconnu" ou "Pas d'événement"
df_prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
df_prev['DAYS_FIRST_DRAWING'].fillna(0, inplace=True) # Imputation par 0 pour le moment (à affiner)


# --- 2. AGRÉGATION PAR CLIENT (SK_ID_CURR) ---

PREV_IGNORE_COLS = ['SK_ID_CURR', 'SK_ID_PREV']
prev_cols = [col for col in df_prev.columns if col not in PREV_IGNORE_COLS]

# Définition des fonctions d'agrégation
prev_num_agg = ['mean', 'min', 'max', 'sum', 'var']
prev_cat_agg = ['mean', 'sum'] # Pour les colonnes issues du OHE

# Structure des agrégations
prev_agg_funcs = {}
for col in prev_cols:
    if df_prev[col].dtype != object:
        # Les colonnes de OHE/compte sont agrégées avec mean (pour le ratio) et sum (pour le compte total)
        if 'FLAG' in col or 'MODE' in col or 'TYPE' in col or 'NAME' in col or 'CODE' in col or 'WEEKDAY' in col or 'HOUR' in col:
            prev_agg_funcs[col] = prev_cat_agg
        # Autres colonnes numériques
        else:
            prev_agg_funcs[col] = prev_num_agg

# Agrégation
prev_grouped = df_prev.groupby('SK_ID_CURR').agg(prev_agg_funcs)

# Aplatir les noms de colonnes
prev_grouped.columns = pd.Index([f'PREV_{col[0]}_{col[1].upper()}' for col in prev_grouped.columns.tolist()])

print(f"Previous Application aggregation done. Aggregated shape: {prev_grouped.shape}")

# --- 3. FUSION DANS LES DATAFRAMES PRINCIPAUX ---

# Fusion avec df_train
df_train = df_train.merge(prev_grouped, how='left', on='SK_ID_CURR')
print(f"Merged previous_application features into df_train. New shape: {df_train.shape}")

# Fusion avec df_test
df_test = df_test.merge(prev_grouped, how='left', on='SK_ID_CURR')
print(f"Merged previous_application features into df_test. New shape: {df_test.shape}")

# Mise à jour des DataFrames
dataframes['train'] = df_train
dataframes['test'] = df_test

print("\nFeature Engineering for Previous Application completed.")
print(f"Total columns in df_train: {df_train.shape[1]}")


--- Starting Feature Engineering: Previous Application ---
Processing previous_application...


C:\Users\maill\AppData\Local\Temp\ipykernel_16988\2199870853.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
C:\Users\maill\AppData\Local\Temp\ipykernel_16988\2199870853.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

Previous Application aggregation done. Aggregated shape: (338857, 443)
Merged previous_application features into df_train. New shape: (307511, 828)
Merged previous_application features into df_test. New shape: (48744, 827)

Feature Engineering for Previous Application completed.
Total columns in df_train: 828


enrichissement des donner 3 

In [13]:
# Nécessaire pour les opérations de fusion et d'agrégation
import pandas as pd
import numpy as np

# Récupération des DataFrames mis à jour
dataframes = globals().get('dataframes') 

if dataframes is None or 'installments_payments' not in dataframes:
    print("Error: DataFrames not loaded or installments_payments is missing.")
    exit() 

df_train = dataframes['train']
df_test = dataframes['test']
df_installments = dataframes['installments_payments']

print("\n--- Starting Feature Engineering: Installments Payments ---")

# --- 1. FEATURE CREATION SPÉCIFIQUE ---

# Calcul de l'écart entre la date de paiement prévue et la date de paiement réelle
# (Valeur négative = paiement en avance, positive = paiement en retard)
df_installments['DAYS_DELAY'] = df_installments['DAYS_ENTRY_PAYMENT'] - df_installments['DAYS_INSTALMENT']

# Ratio du montant payé par rapport au montant attendu (si > 1, le client a trop payé)
df_installments['PAYMENT_RATIO'] = df_installments['AMT_PAYMENT'] / df_installments['AMT_INSTALMENT']

# Difference entre le montant payé et le montant attendu
df_installments['PAYMENT_DIFF'] = df_installments['AMT_PAYMENT'] - df_installments['AMT_INSTALMENT']

# Flag pour le paiement en retard (> 0 jour de retard)
df_installments['LATE_PAYMENT_FLAG'] = (df_installments['DAYS_DELAY'] > 0).astype(np.int8)

# --- 2. AGRÉGATION PAR CLIENT (SK_ID_CURR) ---

INSTALL_IGNORE_COLS = ['SK_ID_CURR', 'SK_ID_PREV', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER']
install_cols = [col for col in df_installments.columns if col not in INSTALL_IGNORE_COLS]

# Fonctions d'agrégation pour les colonnes numériques
install_agg_funcs = {
    'DAYS_DELAY': ['mean', 'min', 'max', 'sum', 'std'],
    'PAYMENT_RATIO': ['mean', 'min', 'max', 'var'],
    'PAYMENT_DIFF': ['mean', 'min', 'max', 'sum', 'var'],
    'LATE_PAYMENT_FLAG': ['sum', 'mean'], # Nombre total et fréquence des retards
    'AMT_INSTALMENT': ['mean', 'sum', 'std'],
    'AMT_PAYMENT': ['mean', 'sum', 'std'],
    'SK_ID_PREV': ['nunique'] # Compter le nombre de prêts précédents associés aux versements
}

# Agrégation
install_grouped = df_installments.groupby('SK_ID_CURR').agg(install_agg_funcs)

# Aplatir les noms de colonnes
install_grouped.columns = pd.Index([f'INSTAL_{col[0]}_{col[1].upper()}' for col in install_grouped.columns.tolist()])

print(f"Installments Payments aggregation done. Aggregated shape: {install_grouped.shape}")

# --- 3. FUSION DANS LES DATAFRAMES PRINCIPAUX ---

# Fusion avec df_train
df_train = df_train.merge(install_grouped, how='left', on='SK_ID_CURR')
print(f"Merged installments_payments features into df_train. New shape: {df_train.shape}")

# Fusion avec df_test
df_test = df_test.merge(install_grouped, how='left', on='SK_ID_CURR')
print(f"Merged installments_payments features into df_test. New shape: {df_test.shape}")

# Mise à jour des DataFrames
dataframes['train'] = df_train
dataframes['test'] = df_test

print("\nFeature Engineering for Installments Payments completed.")
print(f"Total columns in df_train: {df_train.shape[1]}")


--- Starting Feature Engineering: Installments Payments ---
Installments Payments aggregation done. Aggregated shape: (339587, 23)
Merged installments_payments features into df_train. New shape: (307511, 851)
Merged installments_payments features into df_test. New shape: (48744, 850)

Feature Engineering for Installments Payments completed.
Total columns in df_train: 851


enrichissement des donner 4

In [14]:
import pandas as pd
import numpy as np

# Récupération des DataFrames mis à jour
dataframes = globals().get('dataframes') 

if dataframes is None or 'POS_CASH_balance' not in dataframes:
    print("Error: DataFrames not loaded or POS_CASH_balance is missing.")
    exit() 

df_train = dataframes['train']
df_test = dataframes['test']
df_pos_cash = dataframes['POS_CASH_balance']

print("\n--- Starting Feature Engineering: POS CASH Balance ---")

# --- 1. PRÉ-TRAITEMENT ---
print("Processing POS_CASH_balance...")

# One-Hot Encoding pour le statut du prêt (NAME_CONTRACT_STATUS)
df_pos_cash = pd.get_dummies(df_pos_cash, columns=['NAME_CONTRACT_STATUS'], dummy_na=False)

# Création d'une colonne indiquant les retards (Status 1 à 5)
status_cols = [col for col in df_pos_cash.columns if 'NAME_CONTRACT_STATUS_' in col]
if status_cols:
    df_pos_cash['LATE_PAYMENT_POS_CASH'] = df_pos_cash[status_cols].sum(axis=1)
    df_pos_cash['LATE_PAYMENT_POS_CASH'] = df_pos_cash['LATE_PAYMENT_POS_CASH'].clip(upper=1)
else:
    df_pos_cash['LATE_PAYMENT_POS_CASH'] = 0  # Aucun retard si la colonne n'existe pas

# --- 2. AGRÉGATION PAR CLIENT (SK_ID_CURR) ---
POS_CASH_IGNORE_COLS = ['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE']
pos_cash_cols = [col for col in df_pos_cash.columns if col not in POS_CASH_IGNORE_COLS]

# Fonctions d'agrégation
pos_cash_agg_funcs = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'CNT_INSTALMENT': ['mean', 'sum', 'min', 'max', 'std'],
    'CNT_INSTALMENT_FUTURE': ['mean', 'sum', 'min', 'max', 'std'],
    'SK_DPD': ['mean', 'min', 'max', 'sum'],
    'SK_DPD_DEF': ['mean', 'min', 'max', 'sum'],
    'LATE_PAYMENT_POS_CASH': ['mean', 'sum'],
}

# Ajout des colonnes OHE existantes
for col in status_cols:
    pos_cash_agg_funcs[col] = ['mean', 'sum']

# Agrégation
pos_cash_grouped = df_pos_cash.groupby('SK_ID_CURR').agg(pos_cash_agg_funcs)

# Aplatir les noms de colonnes
pos_cash_grouped.columns = pd.Index([f'POS_{col[0]}_{col[1].upper()}' for col in pos_cash_grouped.columns.tolist()])

print(f"POS CASH Balance aggregation done. Aggregated shape: {pos_cash_grouped.shape}")

# --- 3. FUSION DANS LES DATAFRAMES PRINCIPAUX ---
df_train = df_train.merge(pos_cash_grouped, how='left', on='SK_ID_CURR')
print(f"Merged POS_CASH_balance features into df_train. New shape: {df_train.shape}")

df_test = df_test.merge(pos_cash_grouped, how='left', on='SK_ID_CURR')
print(f"Merged POS_CASH_balance features into df_test. New shape: {df_test.shape}")

# Mise à jour des DataFrames
dataframes['train'] = df_train
dataframes['test'] = df_test

print("\nFeature Engineering for POS CASH Balance completed.")
print(f"Total columns in df_train: {df_train.shape[1]}")



--- Starting Feature Engineering: POS CASH Balance ---
Processing POS_CASH_balance...
POS CASH Balance aggregation done. Aggregated shape: (337252, 42)
Merged POS_CASH_balance features into df_train. New shape: (307511, 893)
Merged POS_CASH_balance features into df_test. New shape: (48744, 892)

Feature Engineering for POS CASH Balance completed.
Total columns in df_train: 893


enrichissemnt donner 5

In [15]:
# Nécessaire pour les opérations de fusion et d'agrégation
import pandas as pd
import numpy as np

# Récupération des DataFrames mis à jour
dataframes = globals().get('dataframes') 

if dataframes is None or 'credit_card_balance' not in dataframes:
    print("Error: DataFrames not loaded or credit_card_balance is missing.")
    exit() 

df_train = dataframes['train']
df_test = dataframes['test']
df_cc_balance = dataframes['credit_card_balance']

print("\n--- Starting Feature Engineering: Credit Card Balance (FINAL FILE) ---")

# --- 1. PRÉ-TRAITEMENT ET FEATURE CREATION ---

print("Processing credit_card_balance...")

# One-Hot Encoding
df_cc_balance = pd.get_dummies(df_cc_balance, columns=['NAME_CONTRACT_STATUS'], dummy_na=False)

# Remplacer les valeurs infinies (souvent le résultat de divisions par zéro dans les ratios)
df_cc_balance = df_cc_balance.replace([np.inf, -np.inf], np.nan)
# Imputer les NaN par 0 est souvent une bonne première approche pour ce fichier
df_cc_balance.fillna(0, inplace=True) 

# Création de quelques ratios clés
# Ratio d'utilisation du crédit (très prédictif)
df_cc_balance['LIMIT_USE'] = df_cc_balance['AMT_BALANCE'] / df_cc_balance['AMT_CREDIT_LIMIT_ACTUAL']
df_cc_balance['LIMIT_USE'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_cc_balance['LIMIT_USE'].fillna(0, inplace=True)

# Ratio Paiement / Paiement minimum (important : si < 1, le client n'a pas respecté le minimum)
df_cc_balance['PAYMENT_DIV_MIN'] = df_cc_balance['AMT_PAYMENT_CURRENT'] / df_cc_balance['AMT_INST_MIN_REGULARITY']
df_cc_balance['PAYMENT_DIV_MIN'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_cc_balance['PAYMENT_DIV_MIN'].fillna(0, inplace=True)

# Jours de retard flag
df_cc_balance['CC_LATE_PAYMENT'] = (df_cc_balance['SK_DPD'] > 0).astype(np.int8)


# --- 2. AGRÉGATION PAR CLIENT (SK_ID_CURR) ---

CC_IGNORE_COLS = ['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE']
cc_cols = [col for col in df_cc_balance.columns if col not in CC_IGNORE_COLS]

# Fonctions d'agrégation
cc_agg_funcs = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'AMT_BALANCE': ['mean', 'min', 'max', 'sum', 'std'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'min', 'max'],
    'AMT_DRAWINGS_CURRENT': ['mean', 'sum', 'max'],
    'AMT_INST_MIN_REGULARITY': ['mean', 'min', 'max', 'sum'],
    'AMT_PAYMENT_TOTAL_CURRENT': ['mean', 'sum', 'max', 'std'],
    'LIMIT_USE': ['mean', 'min', 'max', 'std'],
    'PAYMENT_DIV_MIN': ['mean', 'min', 'max', 'std'],
    'CC_LATE_PAYMENT': ['mean', 'sum'],
    'SK_DPD': ['mean', 'max', 'sum'],
}

# Ajout des colonnes OHE
for col in df_cc_balance.columns:
    if 'NAME_CONTRACT_STATUS_' in col:
        cc_agg_funcs[col] = ['mean', 'sum']

# Agrégation
cc_grouped = df_cc_balance.groupby('SK_ID_CURR').agg(cc_agg_funcs)

# Aplatir les noms de colonnes
cc_grouped.columns = pd.Index([f'CC_{col[0]}_{col[1].upper()}' for col in cc_grouped.columns.tolist()])

print(f"Credit Card Balance aggregation done. Aggregated shape: {cc_grouped.shape}")

# --- 3. FUSION DANS LES DATAFRAMES PRINCIPAUX ---

# Fusion avec df_train
df_train = df_train.merge(cc_grouped, how='left', on='SK_ID_CURR')
print(f"Merged credit_card_balance features into df_train. New shape: {df_train.shape}")

# Fusion avec df_test
df_test = df_test.merge(cc_grouped, how='left', on='SK_ID_CURR')
print(f"Merged credit_card_balance features into df_test. New shape: {df_test.shape}")

# Mise à jour des DataFrames
dataframes['train'] = df_train
dataframes['test'] = df_test

print("\nFeature Engineering for Credit Card Balance completed.")
print(f"Total columns in df_train: {df_train.shape[1]}")


--- Starting Feature Engineering: Credit Card Balance (FINAL FILE) ---
Processing credit_card_balance...


C:\Users\maill\AppData\Local\Temp\ipykernel_16988\2183741901.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cc_balance['LIMIT_USE'].replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\maill\AppData\Local\Temp\ipykernel_16988\2183741901.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

Credit Card Balance aggregation done. Aggregated shape: (103558, 50)
Merged credit_card_balance features into df_train. New shape: (307511, 943)
Merged credit_card_balance features into df_test. New shape: (48744, 942)

Feature Engineering for Credit Card Balance completed.
Total columns in df_train: 943


ETAPE 2 TRAITEMNT ML FLOW 

In [16]:
import mlflow
import mlflow.lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

# Features et target
X = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y = df_train['TARGET']

# Split train/validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [28]:
import pandas as pd
import mlflow
import mlflow.lightgbm
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

# --- 1. Chargement des données ---
data_path = "C:/Users/maill/OneDrive/Bureau/majeur_ia/dataenginnering/projet/home-credit-default-risk/application_train.csv"
df_train = pd.read_csv(data_path)

# Features / cible
X = df_train.drop(columns=['SK_ID_CURR', 'TARGET'])
y = df_train['TARGET']

# Identifier les colonnes numériques et catégorielles
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Split train / validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Conversion types pour LightGBM
X_train[num_cols] = X_train[num_cols].astype(np.float32)
X_val[num_cols] = X_val[num_cols].astype(np.float32)

# Conversion colonnes catégorielles en "category"
X_train[cat_cols] = X_train[cat_cols].astype("category")
X_val[cat_cols] = X_val[cat_cols].astype("category")

y_train = y_train.astype(np.int8)
y_val = y_val.astype(np.int8)

# --- 2. Configurer MLflow ---
mlflow.set_tracking_uri("file:///C:/Users/maill/OneDrive/Bureau/mlruns")
mlflow.set_experiment("home_credit_default_risk")

# --- 3. Entraînement LightGBM avec MLflow ---
with mlflow.start_run(run_name="lightgbm_baseline"):
    # Définition du modèle
    model = LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        learning_rate=0.05,
        num_leaves=31,
        max_depth=-1,
        n_estimators=1000,
        verbosity=-1,
        random_state=42
    )

    # Fit avec early stopping et logs toutes les 100 itérations
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        callbacks=[early_stopping(stopping_rounds=50), log_evaluation(period=100)]
    )

    # Prédictions et AUC
    y_pred = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_pred)
    print(f"Validation AUC: {auc_score:.4f}")

    # Logging manuel hyperparamètres et métrique
    mlflow.log_params(model.get_params())
    mlflow.log_metric("val_auc", auc_score)

    # Log du modèle LightGBM (booster_)
    mlflow.lightgbm.log_model(model.booster_, artifact_path="model")

    print("MLflow run completed and model logged successfully!")


Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.75657	valid_0's binary_logloss: 0.246512
[200]	valid_0's auc: 0.758908	valid_0's binary_logloss: 0.245595
Early stopping, best iteration is:
[190]	valid_0's auc: 0.758976	valid_0's binary_logloss: 0.245578


2025/12/13 16:21:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/13 16:22:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Validation AUC: 0.7590
MLflow run completed and model logged successfully!


PARTIE 3

In [29]:
import optuna
from optuna.integration import LightGBMPruningCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import mlflow
import mlflow.lightgbm
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# --- 1. RE-CHARGEMENT ET PRÉPARATION DES DONNÉES (Pour être autonome) ---
# Vous avez déjà fait ce chargement, mais on le garde ici si le notebook est relancé
data_path = "C:/Users/maill/OneDrive/Bureau/majeur_ia/dataenginnering/projet/home-credit-default-risk/application_train.csv"
df_train = pd.read_csv(data_path)
X = df_train.drop(columns=['SK_ID_CURR', 'TARGET'])
y = df_train['TARGET']

num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# Split train / validation
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Conversion types pour LightGBM
X_train[num_cols] = X_train[num_cols].astype(np.float32)
X_val[num_cols] = X_val[num_cols].astype(np.float32)
X_train[cat_cols] = X_train[cat_cols].astype("category")
X_val[cat_cols] = X_val[cat_cols].astype("category")
y_train = y_train.astype(np.int8)
y_val = y_val.astype(np.int8)


# --- 2. CONFIGURER MLFLOW ---
# On réutilise votre configuration MLflow
mlflow.set_tracking_uri("file:///C:/Users/maill/OneDrive/Bureau/mlruns")
mlflow.set_experiment("home_credit_default_risk")


# --- 3. FONCTION OBJECTIF OPTUNA (avec Nested MLflow Run) ---

def objective(trial):
    """
    Fonction objectif pour Optuna, trace les résultats dans une run MLflow imbriquée.
    """
    # 1. Définition de l'espace de recherche
    param = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'n_estimators': 1000, 
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': -1,
    }

    # 2. Démarrer la run MLflow imbriquée
    # Le paramètre 'nested=True' permet d'enregistrer cet essai sous la run principale de l'étude
    with mlflow.start_run(run_name=f"trial_{trial.number}", nested=True):
        
        # Loguer les paramètres testés
        mlflow.log_params(param)
        
        # 3. Entraînement du modèle
        model = LGBMClassifier(**param)

        # Callback Optuna pour l'élagage (pruning) intelligent
        pruning_callback = LightGBMPruningCallback(trial, 'auc', valid_name='valid_0')

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric='auc',
            callbacks=[early_stopping(stopping_rounds=50, verbose=-1), pruning_callback]
        )

        # 4. Évaluation et Log
        y_pred = model.predict_proba(X_val)[:, 1]
        auc_score = roc_auc_score(y_val, y_pred)
        
        mlflow.log_metric("val_auc", auc_score)
        
        # Log du modèle si c'est le meilleur essai de l'étude (facultatif mais utile)
        # On ne logue que si l'AUC est significativement meilleure que la baseline (0.7590)
        if auc_score > 0.7600:
             mlflow.lightgbm.log_model(
                 model.booster_, 
                 artifact_path=f"model_trial_{trial.number}",
                 registered_model_name="HomeCredit_LGBM_Optimized"
             )
        
        # Retourner la métrique à Optuna
        return auc_score

# --- 4. EXÉCUTION DE L'ÉTUDE OPTUNA (Main MLflow Run) ---

N_TRIALS = 50 # Nombre d'essais (augmentez si vous avez plus de temps)
STUDY_NAME = "LGBM_Optimization_Optuna"

# Démarrer la run principale pour l'étude entière
with mlflow.start_run(run_name=STUDY_NAME) as study_run:
    
    # Tags pour l'étude complète
    mlflow.set_tag("Optimization_Tool", "Optuna")
    mlflow.set_tag("Optimization_Phase", "4.2_Hyperparameter_Tuning")
    mlflow.log_param("N_TRIALS", N_TRIALS)
    mlflow.log_param("Validation_Strategy", "Holdout_Split_20pct")
    
    # Créer le study Optuna
    study = optuna.create_study(direction='maximize', study_name=STUDY_NAME)
    
    print(f"\nStarting {N_TRIALS} optimization trials...")
    study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

    # Récupération et log des meilleurs résultats
    best_params = study.best_params
    best_value = study.best_value
    
    # Loguer les résultats finaux dans la main run
    mlflow.log_metric("best_auc_found", best_value)
    mlflow.log_params({f"final_{k}": v for k, v in best_params.items()})

    print("\n--- Optimization Complete ---")
    print(f"Best AUC found: {best_value:.4f}")
    print(f"Best parameters: {best_params}")

[I 2025-12-13 16:29:35,297] A new study created in memory with name: LGBM_Optimization_Optuna



Starting 50 optimization trials...


  0%|          | 0/50 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[273]	valid_0's auc: 0.761202	valid_0's binary_logloss: 0.244932


2025/12/13 16:30:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/13 16:30:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\maill\OneDrive\Bureau\majeur_ia\dataenginnering\projet\.venv\lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance.
  return FileStore(store_uri)
Successfully registered model 'HomeCredit_LGBM_Optimized'.
Created version '1' of model 'HomeCredit_LGBM_Optimized'.
Best trial: 0. Best value: 0.761202:   2%|▏         | 1/50 [00:37<30:33, 37.42s/it]

...

[I 2025-12-13 16:49:49,186] Trial 47 pruned. Trial was pruned at iteration 0.


Best trial: 2. Best value: 0.761422:  98%|█████████▊| 49/50 [20:35<00:22, 22.26s/it]

Training until validation scores don't improve for 50 rounds
[I 2025-12-13 16:50:10,430] Trial 48 pruned. Trial was pruned at iteration 6.


Best trial: 2. Best value: 0.761422: 100%|██████████| 50/50 [20:57<00:00, 25.14s/it]

[I 2025-12-13 16:50:32,406] Trial 49 pruned. Trial was pruned at iteration 0.

--- Optimization Complete ---
Best AUC found: 0.7614
Best parameters: {'learning_rate': 0.09568614915227072, 'num_leaves': 33, 'max_depth': 3, 'min_child_samples': 82, 'subsample': 0.996852935670556, 'colsample_bytree': 0.9999538384549915, 'reg_alpha': 4.529034056419769e-05, 'reg_lambda': 1.3491337131447425e-07}


PARTIE 4

In [30]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.lightgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

# --- 1. CONFIGURATION ET RÉCUPÉRATION DES DONNÉES ---
mlflow.set_tracking_uri("file:///C:/Users/maill/OneDrive/Bureau/mlruns")
mlflow.set_experiment("home_credit_default_risk")

# Chemin des données (à ajuster si besoin)
train_path = "C:/Users/maill/OneDrive/Bureau/majeur_ia/dataenginnering/projet/home-credit-default-risk/application_train.csv"
test_path = "C:/Users/maill/OneDrive/Bureau/majeur_ia/dataenginnering/projet/home-credit-default-risk/application_test.csv"
sample_submission_path = "C:/Users/maill/OneDrive/Bureau/majeur_ia/dataenginnering/projet/home-credit-default-risk/sample_submission.csv"

# Chargement des données
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
submission = pd.read_csv(sample_submission_path)

# Préparation (doit être la même que pour l'optimisation)
X_train_full = df_train.drop(columns=['SK_ID_CURR', 'TARGET'])
y_train_full = df_train['TARGET']
X_test = df_test.drop(columns=['SK_ID_CURR'])

# Identifier les colonnes et types
num_cols = X_train_full.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_train_full.select_dtypes(include=['object']).columns

# Appliquer la même transformation de types pour LightGBM
for df in [X_train_full, X_test]:
    df[num_cols] = df[num_cols].astype(np.float32)
    for col in cat_cols:
        # Gère les catégories manquantes dans le test set
        if col in df.columns:
            df[col] = df[col].astype("category")

y_train_full = y_train_full.astype(np.int8)

# --- 2. RÉCUPÉRATION DES MEILLEURS PARAMÈTRES OPTUNA ---

# Nous utilisons ici les meilleurs paramètres que vous avez trouvés
BEST_PARAMS = {
    'learning_rate': 0.09568614915227072, 
    'num_leaves': 33, 
    'max_depth': 3, 
    'min_child_samples': 82, 
    'subsample': 0.996852935670556, 
    'colsample_bytree': 0.9999538384549915, 
    'reg_alpha': 4.529034056419769e-05, 
    'reg_lambda': 1.3491337131447425e-07,
    # Paramètres additionnels LightGBM
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'n_estimators': 1000, # Laisser élevé pour que l'early stopping fonctionne
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1,
}

# --- 3. ENTRAÎNEMENT FINAL AVEC MLflow ---
print("--- Starting Final Model Training (Partie 4 / Étape 4.3) ---")

with mlflow.start_run(run_name="final_optimized_model"):
    
    # Ajout de tags pour identifier cette run
    mlflow.set_tag("Model_Phase", "4.3_Final_Training")
    mlflow.set_tag("Optimization_Source", "Optuna_Trial_2")
    mlflow.log_param("Train_Data_Size", len(X_train_full))
    
    # Initialisation du modèle avec les meilleurs paramètres
    final_model = LGBMClassifier(**BEST_PARAMS)
    
    # Entraînement sur la totalité des données d'entraînement (sans validation split ici)
    # Dans un vrai cas, on ferait du K-Fold ici, mais pour la simplicité, on fait un fit direct.
    final_model.fit(
        X_train_full, y_train_full,
    )
    
    # Prédictions sur le jeu de test
    test_preds = final_model.predict_proba(X_test)[:, 1]

    # --- 4. CRÉATION DU FICHIER DE SOUMISSION ---
    submission['TARGET'] = test_preds
    submission_filename = f'submission_optimized_auc_0.7614_{pd.Timestamp.now().strftime("%Y%m%d_%H%M")}.csv'
    submission.to_csv(submission_filename, index=False)
    
    print(f"\nSubmission file created: {submission_filename}")
    
    # --- 5. LOGGING FINAL DANS MLflow ---
    # Loguer les paramètres complets du modèle final
    mlflow.log_params(final_model.get_params())
    
    # Loguer le modèle final lui-même
    mlflow.lightgbm.log_model(
        final_model.booster_, 
        artifact_path="final_model_artifact",
        registered_model_name="HomeCredit_LGBM_FINAL"
    )

    # Loguer le fichier de soumission comme artefact
    mlflow.log_artifact(submission_filename)

print("\n--- Final Model Training and Submission Completed ---")
print("Votre fichier de soumission est prêt à être envoyé à Kaggle !")

--- Starting Final Model Training (Partie 4 / Étape 4.3) ---


2025/12/13 17:11:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/13 17:11:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Submission file created: submission_optimized_auc_0.7614_20251213_1711.csv

--- Final Model Training and Submission Completed ---
Votre fichier de soumission est prêt à être envoyé à Kaggle !


Successfully registered model 'HomeCredit_LGBM_FINAL'.
Created version '1' of model 'HomeCredit_LGBM_FINAL'.
